# Импорты

In [2]:
    from ibapi.client import EClient
    from ibapi.wrapper import EWrapper
    from ibapi.contract import Contract
    from threading import Timer
    import pandas as pd
    from datetime import datetime, timezone, timedelta
    import psycopg2

# Портфель

In [8]:
    # Настройка вывода таблиц
    pd.options.display.width = None
    pd.options.display.max_columns = None
    # pd.set_option('display.max_rows', 3000)
    pd.set_option('display.max_columns', 3000)

    ## Объявление переменных
    column_account = ['AccountCode', 'Currency', 'AvailableFunds', 'TotalCashBalance', 'UnrealizedPnL', 'BuyingPower',
                      'Cushion',
                      'DayTradesRemaining', 'ExcessLiquidity', 'FullAvailableFunds', 'FullExcessLiquidity',
                      'GrossPositionValue', 'InitMarginReq',
                      'Leverage-S', 'LookAheadAvailableFunds', 'LookAheadExcessLiquidity', 'LookAheadInitMarginReq',
                      'LookAheadMaintMarginReq', 'NetDividend',
                      'NetLiquidation', 'OptionMarketValue', 'PreviousDayEquityWithLoanValue', 'StockMarketValue',
                      'TBillValue', 'TBondValue', 'WarrantValue',
                      'ExchangeRate']

    dict_option_db = {'open_option_id': [], 'ticker': [], 'option_type': [],
                      'option_direct': [], 'strike_price': [], 'average_cost': [], 'date_expiration': [], 'days_to_exp': [],
                      'size_unit': [], 'money_unit': [], 'create_dttm': [], 'broker_name': [],
                      'comment': []}

    dict_portfolio = {"Symbol": [], "SecType": [], "Exchange": [], "Position": [], "MarketPrice": [], "MarketValue": [],
                      "AverageCost": [], "UnrealizedPNL": [], "RealizedPNL": [], "AccountName": []}


    class TestApp(EWrapper, EClient):
        def __init__(self):
            EClient.__init__(self, self)

        def error(self, reqId, errorCode, errorString):
            print("Error: ", reqId, " ", errorCode, " ", errorString)

        def nextValidId(self, orderId):
            self.start()

#         def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
#             print('accountSummary', reqId, account, tag, value, currency)

        def updatePortfolio(self, contract: Contract, position: float, marketPrice: float, marketValue: float,
                            averageCost: float, unrealizedPNL: float, realizedPNL: float, accountName: str):
            # print(contract.secType)
            if contract.secType == 'OPT':
                option_direct = 'Long' if averageCost > 0 else 'Short'
                # print(contract.lastTradeDateOrContractMonth)
                days_to_exp = (datetime.strptime(contract.lastTradeDateOrContractMonth,
                                                 "%Y%m%d").date() - datetime.now().date()).days
                date_expiration = datetime.strptime(contract.lastTradeDateOrContractMonth,
                                                    "%Y%m%d").date()
                dict_add_db = {'open_option_id': contract.conId, 'ticker': contract.symbol,
                               'option_type': contract.right,
                               'option_direct': option_direct, 'strike_price': contract.strike,
                               'average_cost': averageCost / 100,
                               'date_expiration': date_expiration,
                               'days_to_exp': days_to_exp, 'size_unit': position, 'money_unit': averageCost * position,
                               'create_dttm': datetime.now().date(), 'broker_name':'IB' ,'comment': 'test'}

                for i in dict_option_db.keys():
                    list_ = dict_option_db[i]
                    list_.append(dict_add_db[i])
                    dict_option_db[i] = list_

        def updateAccountValue(self, key: str, val: str, currency: str, accountName: str):
            if (key in column_account) and (currency == 'USD'):
                dict_account[key] = [val]
#                 print("UpdateAccountValue. Key:", key, "Value:", val, "Currency:", currency, "AccountName:", accountName)

#         def updateAccountTime(self, timeStamp: str):
#             print("UpdateAccountTime. Time:", timeStamp)

#         def accountDownloadEnd(self, accountName: str):
#             print("AccountDownloadEnd. Account:", accountName)

        def start(self):
            # Account number can be omitted when using reqAccountUpdates with single account structure
            # print(1)
            self.reqAccountUpdates(True, "")
            # self.reqPositions(True, "U10762468");
            # self.reqAccountSummary(9004, "All", "$LEDGER:ALL")

        def stop(self):
            self.reqAccountUpdates(False, "")
            self.done = True
            self.disconnect()


    def main():
        app = TestApp()
        app.nextOrderId = 0
        app.connect("127.0.0.1", 7496, 1253)
        Timer(2, app.stop).start()
        app.run()


    data_portfolio = []
    data_account = []
    dict_account = {}

    if __name__ == "__main__":
        main()


    def create_df_from_dict(dict_data: dict):
        """Функция создает датафрейм из словаря"""
        df = pd.DataFrame.from_dict(dict_data)
        return df


    df_portfolio = create_df_from_dict(dict_option_db)
    display(df_portfolio)
    df_account = create_df_from_dict(dict_account)
    display(df_account)


Error:  -1   2104   Соединение с базой рыночных данных исправно:hfarm
Error:  -1   2104   Соединение с базой рыночных данных исправно:eufarmnj
Error:  -1   2104   Соединение с базой рыночных данных исправно:cashfarm
Error:  -1   2104   Соединение с базой рыночных данных исправно:usfuture
Error:  -1   2104   Соединение с базой рыночных данных исправно:usfarm.nj
Error:  -1   2104   Соединение с базой рыночных данных исправно:jfarm
Error:  -1   2104   Соединение с базой рыночных данных исправно:eufarm
Error:  -1   2104   Соединение с базой рыночных данных исправно:usfarm
Error:  -1   2104   Соединение с базой рыночных данных исправно:usopt
Error:  -1   2106   Соединение с базой данных HMDS исправно:euhmds
Error:  -1   2106   Соединение с базой данных HMDS исправно:fundfarm
Error:  -1   2106   Соединение с базой данных HMDS исправно:ushmds
Error:  -1   2158   Соединение с базой данных "sec-def" исправно:secdefeu


,open_option_id,ticker,option_type,option_direct,strike_price,average_cost,date_expiration,days_to_exp,size_unit,money_unit,create_dttm,broker_name,comment
0,625032700,ZIM,P,Long,7.5,0.227991,2023-10-20,66,1.0,22.7991,2023-08-15,IB,test
1,614934310,ZIM,C,Long,12.5,1.244490,2023-10-20,66,4.0,497.7962,2023-08-15,IB,test
2,614934632,ZIM,P,Long,12.5,1.386191,2023-10-20,66,1.0,138.6191,2023-08-15,IB,test
3,587974054,ZIM,C,Long,12.5,2.607991,2025-01-17,521,1.0,260.7991,2023-08-15,IB,test
4,584901499,ZIM,C,Long,20.0,1.377991,2025-01-17,521,1.0,137.7991,2023-08-15,IB,test


,AvailableFunds,BuyingPower,Currency,ExcessLiquidity,ExchangeRate,FullAvailableFunds,FullExcessLiquidity,GrossPositionValue,InitMarginReq,LookAheadAvailableFunds,LookAheadExcessLiquidity,LookAheadInitMarginReq,LookAheadMaintMarginReq,NetDividend,NetLiquidation,OptionMarketValue,PreviousDayEquityWithLoanValue,StockMarketValue,TBillValue,TBondValue,TotalCashBalance,UnrealizedPnL,WarrantValue
0,206.03,206.03,USD,376.71,1.00,206.03,376.71,1511.39,227.58,206.03,376.71,227.58,56.89,0.00,1717.42,1283.81,440.33,227.58,0.00,0.00,206.03,227.74,0.00
